In [14]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.plot import * 
from rasterio.mask import * 
from matplotlib import pyplot
import numpy as np
from shapely.geometry import Point
from shapely.geometry import box
import requests, json
import pickle 

from time import sleep

import contextily as ctx
import osmnx as ox

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
import rasterio

from rasterio.plot import show

from rasterio.plot import show_hist

from rasterio.mask import mask

from shapely.geometry import box

import geopandas as gpd

from fiona.crs import from_epsg

import pycrs

In [16]:
#opening population file
population_file = "./data/population_phl_2018-10-01.tif"
data=rasterio.open(population_file)

RasterioIOError: ./data/population_phl_2018-10-01.tif: No such file or directory

In [ ]:
#opening boundary file 
fname="./data/Regions.geojson"
df=gpd.read_file(fname)

In [ ]:
NCR=df.loc[df.ADM1_EN=="National Capital Region",:].reset_index(drop=True)
bounds=NCR.bounds
minx=bounds.loc[0,"minx"]

In [ ]:
#saving bbox
from shapely.geometry import box

bounds = NCR.bounds
bbox = box(
    minx = bounds.loc[0,"minx"],
    miny = bounds.loc[0,"miny"],
    maxx = bounds.loc[0,"maxx"],
    maxy = bounds.loc[0,"maxy"]
)

In [ ]:
geo=gpd.GeoDataFrame({'geometry':bbox}, index=[0],crs=from_epsg(4326))
geo=geo.to_crs(crs=data.crs.data)
_plot_df(geo)

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
coords= getFeatures(geo)
print(coords)
out_img, out_transform = mask(data, shapes=coords, crop=True)

out_meta=data.meta.copy()
print(out_meta)

epsg_code = int(data.crs.data['init'][5:])
print(epsg_code)

out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
                  )

In [ ]:
out_tif="./data/clipped_NCRPh.tif"
with rasterio.open(out_tif, "w", **out_meta) as dest:
              dest.write(out_img)

In [ ]:
clipped=rasterio.open(out_tif)
show((clipped,1),cmap='terrain')